In [59]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordEpisodeStatistics
gymlogger.set_level(40) #error only
from torch import nn
import torch
from torch.nn import functional as F
import numpy as np
import pandas as pd
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from base64 import b64encode

from IPython import display as ipythondisplay
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [61]:
def show_video(episode):
  mp4 = open(f'/content/videos/0.25.2/rl-video-episode-{episode}.mp4','rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  ipythondisplay.display(HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

In [78]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False, render_mode = "rgb_array")

# env = gym.wrappers.RecordEpisodeStatistics(env)
# env = gym.wrappers.RecordVideo(env, f"videos/{gym.__version__}")

In [63]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode = "rgb_array")

In [79]:
class QLearningTable:
  def __init__(self, actions, terminal, learning_rate=0.01, reward_decay=0.9, e_greedy=0.1):
    self.actions = actions
    self.lr = learning_rate
    self.gamma = reward_decay
    self.epsilon = e_greedy
    self.q_table = pd.DataFrame(columns = self.actions, dtype=np.float64)
    self.terminal = terminal

  def choose_action(self, observation, is_test):
    self.check_state_exist(observation)


    if np.random.uniform()>self.epsilon or is_test==True:
      state_action = self.q_table.loc[observation, :]
      action = np.random.choice(state_action[state_action == np.max(state_action)].index)
    else:
      action = np.random.choice(self.actions)
    return action

  def learn(self, s, a, r, s_):
    self.check_state_exist(s_)
    q_predict = self.q_table.loc[s, a]
    if s_ != self.terminal:
      q_target = r+self.gamma*self.q_table.loc[s_, :].max()
    else:
      q_target = r
    self.q_table.loc[s, a] += self.lr*(q_target-q_predict)
    # print(self.q_table.loc[s, a])



  def check_state_exist(self, state):
    if state not in self.q_table.index:
      self.q_table = self.q_table.append(
          pd.Series(
              [0]*len(self.actions),
              index = self.q_table.columns,
              name = state
          )
      )

In [80]:
RL = QLearningTable(range(4), 15, learning_rate=0.5, e_greedy=0.1)

In [81]:
success = 0
total_episode = 500
distinct_rewards = []
for episode in range(total_episode):
  observation = env.reset()
  while True:
    env.render()
    action = RL.choose_action(observation, False)
    observation_, rewards, done, infos = env.step(action)
    if rewards not in distinct_rewards:
      distinct_rewards.append(rewards)
    RL.learn(observation, action, rewards, observation_)
    observation = observation_
    # print(RL.q_table)
    if done:
      if rewards == 1:
        # print("success")
        # print(RL.q_table)
        success += 1
      break
print(success/total_episode)

<ipython-input-79-e07600b93f61>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
<ipython-input-79-e07600b93f61>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
<ipython-input-79-e07600b93f61>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
<ipython-input-79-e07600b93f61>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.q_table = self.q_table.append(
<ipython-input-79-e07600b93f61>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s

0.71


In [56]:
distinct_rewards

[0.0, 1.0]

In [73]:
RL.q_table

,0,1,2,3
0,0.112198,0.043023,0.068437,0.067297
4,0.141351,0.049274,0.021448,0.027436
8,0.091130,0.102436,0.142984,0.122414
9,0.095401,0.098309,0.109921,0.096627
13,0.100386,0.213789,0.224236,0.185293
14,0.393684,0.356745,0.396030,0.339721
12,0.000000,0.000000,0.000000,0.000000
1,0.002235,0.017254,0.002413,0.022405
2,0.012355,0.015425,0.012922,0.015930
6,0.003988,0.100941,0.005434,0.000972


In [82]:
#test
test_success = 0
test_total_episode = 1000
for episode in range(test_total_episode):
  observation = env.reset()
  while True:
    env.render()
    action = RL.choose_action(observation, True)
    observation_, rewards, done, infos = env.step(action)
    observation = observation_
    if done:
      if observation == 15:
        test_success += 1
      break
print(test_success/test_total_episode)

1.0


In [85]:
RL.q_table

,0,1,2,3
0,0.530403,0.000000,0.59049,0.531126
4,0.000000,0.000000,0.00000,0.000000
8,0.000000,0.000000,0.36379,0.000000
12,0.000000,0.000000,0.00000,0.000000
1,0.531182,0.000000,0.65610,0.590333
5,0.000000,0.000000,0.00000,0.000000
9,0.000000,0.809803,0.00000,0.000000
10,0.726966,0.900000,0.00000,0.728813
13,0.000000,0.569111,0.90000,0.170481
14,0.808668,0.891211,1.00000,0.803672
